In [1]:
from __future__ import annotations
import requests
from pydantic import TypeAdapter
from src.main import fetch_api_pokemons, fetch_firestore_pokemons
from src.models import ApiPokemon, Pokemon

In [2]:
raw_api_pokemons = fetch_api_pokemons()

In [3]:
api_pokemons = {
    pokemon_id: Pokemon.from_api_pokemon(api_pokemon) for pokemon_id, api_pokemon in raw_api_pokemons.items()
}

In [4]:
for pokemon_id, api_pokemon in api_pokemons.items():
    for ev_id in api_pokemon.next_evolve_api_ids:
        api_pokemons[ev_id].add_prev_evolve_ids(pokemon_id)

In [5]:
current_firestore_pokemons = fetch_firestore_pokemons()

In [6]:
new_firestore_pokemons = dict()


for pokemin_id, api_pokemon in api_pokemons.items():
    current_pokemon = current_firestore_pokemons.get(pokemin_id)

    if current_pokemon is None:
        # firestore に存在しないポケモンは作成
        print(f'create {pokemin_id}')
        new_firestore_pokemons[pokemin_id] = api_pokemon
    else:
        # firestore に存在し、差分があるポケモンは更新する
        if current_pokemon.is_updated(api_pokemon):
            print(f'update {pokemin_id}')
            new_firestore_pokemons[pokemin_id] = api_pokemon
            new_firestore_pokemons[pokemin_id].exist = current_pokemon.exist
            new_firestore_pokemons[pokemin_id].exist_shadow = current_pokemon.exist_shadow
            new_firestore_pokemons[pokemin_id].exist_shiny = current_pokemon.exist_shiny


update FARFETCHD
update MR_MIME
update WOOPER
update QWILFISH
update SNEASEL
update LINOONE
update CHERUBI
update CHERRIM
update YAMASK
update DEERLING
update SAWSBUCK
update LITLEO
update PYROAR
update ESPURR
update MEOWSTIC
update ROCKRUFF
update LYCANROC
update TOXEL
update TOXTRICITY
update OBSTAGOON
update PERRSERKER
update SIRFETCHD
update MR_RIME
update RUNERIGUS
update KUBFU
update URSHIFU
create WYRDEER
update SNEASLER
update OVERQWIL
update LECHONK
update OINKOLOGNE
update TANDEMAUS
update MAUSHOLD
update CLODSIRE


In [15]:
current_pokemon.model_dump()

{'pokemon_id': 'MEOWTH',
 'name': 'ニャース',
 'dex_no': 52,
 'form': 'NORMAL',
 'generation': 1,
 'exist': False,
 'exist_shiny': False,
 'exist_shadow': False,
 'primary_type': 'POKEMON_TYPE_NORMAL',
 'secondary_type': None,
 'pokemon_class': None,
 'status': {'stamina': 120, 'attack': 92, 'defense': 78},
 'prev_evolve_api_ids': [],
 'next_evolve_api_ids': ['PERSIAN', 'PERRSERKER']}

In [17]:
api_pokemon.model_dump()

{'pokemon_id': 'MEOWTH',
 'name': 'ニャース',
 'dex_no': 52,
 'form': 'NORMAL',
 'generation': 1,
 'exist': False,
 'exist_shiny': False,
 'exist_shadow': False,
 'primary_type': 'POKEMON_TYPE_NORMAL',
 'secondary_type': None,
 'pokemon_class': None,
 'status': {'stamina': 120, 'attack': 92, 'defense': 78},
 'prev_evolve_api_ids': [],
 'next_evolve_api_ids': ['PERRSERKER', 'PERSIAN']}

In [18]:
from src.main import db

In [19]:
for pokemon_id, pokemon in new_firestore_pokemons.items():
    db.collection('pokemons').document(pokemon_id).set(pokemon.model_dump(by_alias=True))
    break